In [1]:
import pandas as pd
from os import listdir
import numpy as np
from os.path import isfile, join
from Bio import SeqIO
HOME_DIR = "../data/pdb_str/"
CHEM_HITS_DIR = "../data/pdb_str/chem_hits/"
PDB_DIR = "../data/pdb_str/fasta/"
CHEM_DIR = "../data/pdb_str/chem/"

In [2]:
clearEmpty = lambda x: [ i for i in x if i != '']
def parse_interval(range):
    segs = [ seg.split("-")  for seg in range.split(",")]
    segs = [ clearEmpty(s) for s in segs ]
    return [ [ int(s[0]), int(s[1].split(":")[0]) ] for s in segs]


def merge_range(elem):
    return ",".join([ ":".join( [str(o) for o in pair] ) for pair in elem])


def scale_hit(pdb_seq, start, end):
    hit_interval = []
    num_char_hit = (end - start + 1)
    num_char_viewed_hit = 0
    num_char_viewed_seq = 0

    for I in pdb_seq:
        start_scaled = max((I[0] + start - 1) - num_char_viewed_seq, I[0])
        end_scaled = min(start_scaled+num_char_hit-1, I[1])
        if start_scaled <= I[1] and num_char_hit > 0:
            hit_interval = hit_interval + [[start_scaled, end_scaled]]
            num_char_hit = num_char_hit - (end_scaled-start_scaled+1)
            num_char_viewed_hit = end_scaled - start_scaled + 1
            num_char_viewed_seq = num_char_viewed_seq + (start_scaled-start)+num_char_viewed_hit
        else:
            num_char_viewed_seq = num_char_viewed_seq +(I[1]-I[0])+1

    return hit_interval

In [3]:
files = [ "".join([CHEM_HITS_DIR, f]) for f in listdir(CHEM_HITS_DIR) ]

In [22]:
pdb_intervals = pd.read_csv("data/cath-b-newest-all", sep=" ", names =["pdb", "version", "cath", "range"])
pdb_intervals = pdb_intervals[["pdb", "range"]]
pdb_intervals.head()

,pdb,range
0,101mA00,0-153:A
1,102lA00,1-162:A
2,102mA00,0-153:A
3,103lA00,1-162:A
4,103mA00,0-153:A


In [35]:
def f10(f):
    _df = pd.read_csv(f)
    return _df[_df.hit_length >= 10]

dfs = [ f10(f) for f in files ]
df = pd.concat(dfs)

NameError: name 'files' is not defined

In [25]:
df.to_csv("../data/pdb_str/chem_hits/all_hits_f10.csv", index=False)

In [4]:
df = pd.read_csv("../data/pdb_str/chem_hits/all_hits_f10.csv")
df.shape

(670222, 4)

In [5]:
aa_filter = pd.read_csv("hit_aa_sample.csv")
aa_filter = aa_filter[aa_filter.hit_length >= 10]
aa_filter["key"] = aa_filter[["x", "y"]].apply(lambda x: "%s_%s" % (x.x, x.y), axis=1)
aa_filter["key_revert"] = aa_filter[["x", "y"]].apply(lambda x: "%s_%s" % (x.y, x.x), axis=1)
aa_filter.head()

,x,y,hit_length,hit,X_ratio,key,key_revert
580,1bg5A03,2zxeA01,12,SFKNMVPQQALV,0.0,1bg5A03_2zxeA01,2zxeA01_1bg5A03
1915,1fp0A00,2if1A00,10,MRGSHHHHHH,0.0,1fp0A00_2if1A00,2if1A00_1fp0A00
2014,1fp0A00,2lfvA00,12,MRGSHHHHHHGS,0.0,1fp0A00_2lfvA00,2lfvA00_1fp0A00
3696,1m6nA05,1tf5A04,85,ENLREIVENMIKSSLERAIAAYTPREELPEEWKLDGLVDLINTTYL...,0.0,1m6nA05_1tf5A04,1tf5A04_1m6nA05
6409,1wgsA00,2ctfA00,10,GSSGSSGEPE,0.0,1wgsA00_2ctfA00,2ctfA00_1wgsA00


In [7]:
df["x_chain"] = df.x.apply(lambda x: x[0:-3])
df["y_chain"] = df.y.apply(lambda x: x[0:-3])

In [8]:
df = df.loc[df[["x_chain"]].drop_duplicates().index]
df = df.loc[df[["y_chain"]].drop_duplicates().index]
df.shape

(3468, 6)

In [9]:
df["key"] = df[["x", "y"]].apply(lambda x: "%s_%s" % (x.x, x.y), axis=1)
df.head(2)

,x,y,hit_length,hit,x_chain,y_chain,key
0,12asA00,16pkA02,10,aaaaaabahs,12as,16pk,12asA00_16pkA02
315,132lA00,1gkgA01,10,wabaahsama,132l,1gkg,132lA00_1gkgA01


In [10]:
df = pd.merge(df, aa_filter[["key", "key_revert"]], on="key", how="left")
df = pd.merge(df, aa_filter[["key", "key_revert"]], left_on="key", right_on="key_revert", how="left")

In [11]:
df = df[df.key_revert_x.isna() & df.key_revert_y.isna()]
df.shape


(3433, 10)

In [12]:
df = df[df.hit_length > 11]

In [13]:
# read fasta
def readPDB(path):
    for record in SeqIO.parse(path, "fasta"):
        pdb = "".join(list(record.seq))
    return pdb

def filter_substring(x, y):
    if x.find(y) > 0 or y.find(x) > 0:
        return False
    else:
        return True

In [14]:
df.loc[:,"x_pdb_seq"] = df["x"].apply(lambda x: readPDB("%s%s.fasta" % (PDB_DIR, x)))
df.loc[:,"x_chem_seq"] = df["x"].apply(lambda x: readPDB("%s%s.fasta" % (CHEM_DIR, x)))
df.loc[:,"y_pdb_seq"] = df["y"].apply(lambda x: readPDB("%s%s.fasta" % (PDB_DIR, x)))
df.loc[:,"y_chem_seq"] = df["y"].apply(lambda x: readPDB("%s%s.fasta" % (CHEM_DIR, x)))
df.loc[:, "x_hit_start"] = df[["x_chem_seq", "hit"]].apply(lambda x: x.x_chem_seq.find(x.hit), axis=1)
df.loc[:, "x_hit_end"]   = df[["x_hit_start", "hit"]].apply(lambda x: x.x_hit_start + len(x.hit), axis=1)
df.loc[:, "y_hit_start"] = df[["y_chem_seq", "hit"]].apply(lambda x: x.y_chem_seq.find(x.hit), axis=1)
df.loc[:, "y_hit_end"]   = df[["y_hit_start", "hit"]].apply(lambda x: x.y_hit_start + len(x.hit), axis=1)
df.head()

,x,y,hit_length,hit,x_chain,y_chain,key_x,key_revert_x,key_y,key_revert_y,x_pdb_seq,x_chem_seq,y_pdb_seq,y_chem_seq,x_hit_start,x_hit_end,y_hit_start,y_hit_end
36,1e5mA01,1e8cA02,12,aaaaaahaaaaa,1e5m,1e8c,1e5mA01_1e8cA02,NaN,NaN,NaN,KKRVVVTGLGAITPIGNTLQDYWQGLMEGRNGIGPITRFDASDQAC...,bbbaaahaaaaahiaamhamsrrmaawsabmaaaiahbrsahsmaw...,PSDNLRLVGVTGTNGKTTTTQLLAQWSQLLGEISAVGTVGNGLLGK...,ihsmabaaaahahmabhhhhmaaamrhmaaasahaaahaamaaaab...,98,110,201,213
47,1fp1D01,1jb0A00,12,maariaaamaaa,1fp1,1jb0,1fp1D01_1jb0A00,NaN,NaN,NaN,QTEDSACLSAMVLTTNLVYPAVLNAAIDLNLFEIIAKATPPGAFMS...,mhsshawahawaahhmaariaaamaaasamarsaaabahiiaarwh...,RVVVDNDPVPTSFEKWAKPGHFDRTLARGPQTTTWIWNLHALAHDF...,baaasmsiaihhrsbrabiabrsbhaabaimhhhrarmabaaabsr...,15,27,102,114
83,1l3aA00,1t3tA04,12,arbhaamhrbis,1l3a,1t3t,1l3aA00_1t3tA04,NaN,NaN,NaN,TPKVFVGYSIYKGKAALTVEPRSPEFSPLDSGAFKLSREGMVMLQF...,hibaraarharbabaaahasibhisrhiashaarbahbsawawamr...,RTTLRVWWAETTWQMQRLRDNPQCADQEHEAKANDTDPGLNVKLSF...,bhhabarrashhrmwmbabsmimwasmsbsabamshsiaamabahr...,154,166,285,297
214,1hjzA00,1i9gA01,12,bahaabsaaaaa,1hjz,1i9g,1hjzA00_1i9gA01,NaN,NaN,NaN,MEVLFEAKVGDITLKLAQGDITQYPAKAIVNAANKRLEHGGGVAYA...,wsaarsabaasahabaamasahmriabaaamaambbasbaaaaara...,TGPFSIGERVQLTDAKGRRYTMSLTPGAEFHTHRGSIAHDAVIGLE...,hairhaasbamahsababbrhwhahiaasrbhbbahaabsaaaaas...,163,175,33,45
218,1ijyA00,1m53A01,12,bshmssaaasab,1ijy,1m53,1ijyA00_1m53A01,NaN,NaN,NaN,ELACQEITVPLCKGIGYEYTYMPNQFNHDTQDEAGLEVHQFWPLVE...,saawmsahaiawbaaarsrhrwimmrmbshmssaaasabmrriaas...,EYPAWWKEAVFYQIYPRSFKDTNDDGIGDIRGIIEKLDYLKSLGID...,sriarrbsaarrmaribhrbshmssaaasabaaasbasrabhaaas...,27,39,19,31


In [51]:
df.loc[:, "x_pdb_fragment"]  = df[["x_pdb_seq", "x_hit_start", "x_hit_end"]].apply(lambda x: x.x_pdb_seq[x.x_hit_start:x.x_hit_end], axis=1)
df.loc[:, "y_pdb_fragment"]  = df[["y_pdb_seq", "y_hit_start", "y_hit_end"]].apply(lambda x: x.y_pdb_seq[x.y_hit_start:x.y_hit_end], axis=1)

In [58]:
df.rename(columns={"hit" : "chem_fragment"})[["x", "y", "hit_length","chem_fragment","x_pdb_fragment","y_pdb_fragment"]]\
.to_csv("chem_hits_sample.csv", index=False)


In [52]:
df_hits = pd.read_csv("chem_hits_sample.csv")
# df_hits[df_hits.x == "1im2A02"]

,x,y,hit_length,chem_fragment,x_pdb_fragment,y_pdb_fragment
0,1e5mA01,1e8cA02,12,aaaaaahaaaaa,IGVLIGTGIGGL,LLLALATLLALG
1,1fp1D01,1jb0A00,12,maariaaamaaa,NLVYPAVLNAAI,QVVWPIVGQGIL
2,1l3aA00,1t3tA04,12,arbhaamhrbis,GWHTAVNSFKPE,VFRTVANSWHPE
3,1hjzA00,1i9gA01,12,bahaabsaaaaa,KGSAVKEVALVI,RGSIAHDAVIGL
4,1ijyA00,1m53A01,12,bshmssaaasab,HDTQDEAGLEVH,KDTNDDGIGDIR
5,1jekA00,1ji0A00,12,aabaabaasaba,IAKGIRILEARV,AIHAIKGIDLKV
6,1k70A01,1k7cA00,13,rhabaabaaahhm,YSVRGGKVIASTQ,FTAKGAKVILSSQ
7,1qo2A00,1qysA00,12,aaaaabarasaa,GVIVGRAFLEGI,AAILIKVFAELG
8,1x72A00,1yd9B00,12,asaahashsaaa,LEGGSIESDGAG,ADIASIDSDAVV
9,1aqiA02,1sqhA01,12,aasarbabraab,LGDLFHIRFAAR,LLDLYKVKFGIR


In [44]:
df[df.x == "1im2A02"]

,x,y,hit_length,hit,x_chain,y_chain,key_x,key_revert_x,key_y,key_revert_y,x_pdb_seq,x_chem_seq,y_pdb_seq,y_chem_seq,x_hit_start,x_hit_end,y_hit_start,y_hit_end
608,1im2A02,1iomA02,13,baasssaabaami,1im2,1iom,1im2A02_1iomA02,NaN,NaN,NaN,LVRQQEIAKNRARAEDVAEERLIDDEAAKLINPEELKQKA,aabmmsaabmbabassaassbaasssaabaamissabmba,GANEAVMRMIQEIGTPERAREWVREKLAKKERIMGMGHRVYKAFDP...,aamsaawbwamsaahisbabsrabsbaabbsbawawabbarbarsi...,20,33,71,84


In [45]:
I = 608
df.loc[I].x_chem_seq

'aabmmsaabmbabassaassbaasssaabaamissabmba'

In [46]:
df.loc[I].x_pdb_seq

'LVRQQEIAKNRARAEDVAEERLIDDEAAKLINPEELKQKA'

In [47]:
df.loc[I].y_chem_seq

'aamsaawbwamsaahisbabsrabsbaabbsbawawabbarbarsibaaaasbaabaaasbbabhbsrmaabaasssaabaamibaarimasrrhaaarhsa'

In [48]:
df.loc[I].y_pdb_seq

'GANEAVMRMIQEIGTPERAREWVREKLAKKERIMGMGHRVYKAFDPRAGVLEKLARLVAEKHGHSKEYQILKIVEEEAGKVLNPRGIYPNVDFYSGVVYSDL'